In [2]:
import hyperopt
import numpy as np
import simpy


from scipy.optimize import minimize
from scipy.stats import expon

In [3]:
def obj(z):
    x = z[0]
    y = z[1]
    return x**2 + y**2 + 2 * x + 3 * y + 1

res = minimize(obj, [0, 0])
print(res)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -2.2499999999999982
        x: [-1.000e+00 -1.500e+00]
      nit: 2
      jac: [-2.980e-08 -5.960e-08]
 hess_inv: [[ 8.462e-01 -2.308e-01]
            [-2.308e-01  6.538e-01]]
     nfev: 9
     njev: 3


In [4]:
res_2 = hyperopt.fmin(fn=obj,
                      algo=hyperopt.tpe.suggest,
                      max_evals=100,
                      rstate=np.random.default_rng(42),
                      space=[hyperopt.hp.normal('x',0,10),
                             hyperopt.hp.normal('y',0,10)])


  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 100/100 [00:00<00:00, 270.40trial/s, best loss: -2.244976618661607]


In [5]:
res_2

{'x': -0.990991143473577, 'y': -1.5703009377069734}

In [6]:
### fast food restaurant with capacity 50 people
### n_1 checkout workers
### n_2 kitchen workers
### customers arrive according to HPP with intensity lam_dt=100
### checkout worker takes an exponential amount of time to take an order lam_st1 = 50
### kitchen takes an exponential amount of time to make an order lam_st2 = 25
### if a customer arrives and the store is full, they leave
### every customer spends $5
### at T=1, everyone gets kicked out of the store
### each worker gets paid $22
### find the optimal values of n_1 and n_2 to maximize profit



lam_dt = 100
lam_st1 = 50
lam_st2 = 25

def simulation(n):

    n_checkout = n[0]
    n_kitchen = n[1]
    n_workers = n_checkout + n_kitchen
    cust_capacity = 50 - n_workers
    rev = []

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            st1 = expon.rvs(scale=1/lam_st1)
            st2 = expon.rvs(scale=1/lam_st2) # generate together to maximum the correlation ?
            # rather than generate seperately
            yield env.timeout(dt)
            if store.count < cust_capacity:
                rqt_st = store.request()
                yield rqt_st
                env.process(service(rqt_st, st1, st2))

    def service(rqt_st, st1, st2):
        rqt1 = checkout.request()
        yield rqt1
        yield env.timeout(st1)
        checkout.release(rqt1)
        rqt2 = kitchen.request()
        yield rqt2
        yield env.timeout(st2)
        kitchen.release(rqt2)
        store.release(rqt_st)
        rev.append(5)

    if cust_capacity <=0:
        return 100_000
    else:
        env = simpy.Environment()
        store = simpy.Resource(env=env, capacity=cust_capacity)
        checkout = simpy.Resource(env=env, capacity=n_checkout)
        kitchen = simpy.Resource(env=env, capacity=n_kitchen)
        env.process(arrivals())
        env.run(until=1)

        return 22 * n_workers - np.sum(rev)


def obj(n):
    np.random.seed(37)
    nsims = 100
    revs = [simulation(n) for i in range(nsims)]
    return np.mean(revs)


res = hyperopt.fmin(fn=obj, 
                    algo=hyperopt.tpe.suggest, 
                    max_evals=50,
                    rstate=np.random.default_rng(42),
                    space=[hyperopt.hp.quniform('n_check', 1, 30, 1),
                           hyperopt.hp.quniform('n_kitchen', 1, 30, 1)])



# maximum correlations

100%|██████████| 50/50 [01:44<00:00,  2.10s/trial, best loss: -273.15]
